# ਟੈਕਸਟ ਵਰਗੀਕਰਨ ਟਾਸਕ

ਇਸ ਮੋਡੀਊਲ ਵਿੱਚ, ਅਸੀਂ **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** ਡੇਟਾਸੈੱਟ ਦੇ ਆਧਾਰ 'ਤੇ ਇੱਕ ਸਧਾਰਨ ਟੈਕਸਟ ਵਰਗੀਕਰਨ ਟਾਸਕ ਸ਼ੁਰੂ ਕਰਾਂਗੇ: ਅਸੀਂ ਖਬਰਾਂ ਦੇ ਸਿਰਲੇਖਾਂ ਨੂੰ 4 ਸ਼੍ਰੇਣੀਆਂ ਵਿੱਚ ਵਰਗਬੱਧ ਕਰਾਂਗੇ: ਵਿਸ਼ਵ, ਖੇਡਾਂ, ਕਾਰੋਬਾਰ ਅਤੇ ਵਿਗਿਆਨ/ਤਕਨਾਲੋਜੀ।

## ਡੇਟਾਸੈੱਟ

ਡੇਟਾਸੈੱਟ ਨੂੰ ਲੋਡ ਕਰਨ ਲਈ, ਅਸੀਂ **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ।


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

ਅਸੀਂ ਹੁਣ `dataset['train']` ਅਤੇ `dataset['test']` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਡੇਟਾਸੈੱਟ ਦੇ ਟ੍ਰੇਨਿੰਗ ਅਤੇ ਟੈਸਟ ਹਿੱਸਿਆਂ ਤੱਕ ਪਹੁੰਚ ਕਰ ਸਕਦੇ ਹਾਂ:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


ਆਓ ਆਪਣੇ ਡੇਟਾਸੈੱਟ ਵਿੱਚੋਂ ਪਹਿਲੀਆਂ 10 ਨਵੀਆਂ ਸੁਰਖੀਆਂ ਪ੍ਰਿੰਟ ਕਰੀਏ:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## ਟੈਕਸਟ ਵੇਕਟਰਾਈਜ਼ੇਸ਼ਨ

ਹੁਣ ਸਾਨੂੰ ਟੈਕਸਟ ਨੂੰ **ਨੰਬਰਾਂ** ਵਿੱਚ ਬਦਲਣਾ ਪਵੇਗਾ ਜੋ ਟੈਂਸਰ ਦੇ ਰੂਪ ਵਿੱਚ ਦਰਸਾਏ ਜਾ ਸਕਣ। ਜੇਕਰ ਅਸੀਂ ਸ਼ਬਦ-ਪੱਧਰ ਦੀ ਪ੍ਰਤੀਨਿਧਤਾ ਚਾਹੁੰਦੇ ਹਾਂ, ਤਾਂ ਸਾਨੂੰ ਦੋ ਗੱਲਾਂ ਕਰਨੀਆਂ ਪੈਣਗੀਆਂ:

* ਇੱਕ **ਟੋਕਨਾਈਜ਼ਰ** ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਟੈਕਸਟ ਨੂੰ **ਟੋਕਨਜ਼** ਵਿੱਚ ਵੰਡੋ।
* ਉਹਨਾਂ ਟੋਕਨਜ਼ ਦੀ ਇੱਕ **ਸ਼ਬਦਾਵਲੀ** ਤਿਆਰ ਕਰੋ।

### ਸ਼ਬਦਾਵਲੀ ਦੇ ਆਕਾਰ ਨੂੰ ਸੀਮਿਤ ਕਰਨਾ

AG News ਡੇਟਾਸੈੱਟ ਦੇ ਉਦਾਹਰਨ ਵਿੱਚ, ਸ਼ਬਦਾਵਲੀ ਦਾ ਆਕਾਰ ਕਾਫ਼ੀ ਵੱਡਾ ਹੈ, 100,000 ਤੋਂ ਵੱਧ ਸ਼ਬਦ। ਆਮ ਤੌਰ 'ਤੇ, ਸਾਨੂੰ ਉਹ ਸ਼ਬਦਾਂ ਦੀ ਲੋੜ ਨਹੀਂ ਹੁੰਦੀ ਜੋ ਟੈਕਸਟ ਵਿੱਚ ਘੱਟ ਹੀ ਮਿਲਦੇ ਹਨ — ਸਿਰਫ ਕੁਝ ਹੀ ਵਾਕਾਂ ਵਿੱਚ ਉਹ ਸ਼ਬਦ ਹੋਣਗੇ, ਅਤੇ ਮਾਡਲ ਉਹਨਾਂ ਤੋਂ ਕੁਝ ਨਹੀਂ ਸਿੱਖੇਗਾ। ਇਸ ਲਈ, ਸ਼ਬਦਾਵਲੀ ਦੇ ਆਕਾਰ ਨੂੰ ਛੋਟੇ ਨੰਬਰ ਤੱਕ ਸੀਮਿਤ ਕਰਨਾ ਸਮਝਦਾਰੀ ਹੈ, ਜੋ ਕਿ ਵੇਕਟਰਾਈਜ਼ਰ ਕੰਸਟਰਕਟਰ ਨੂੰ ਇੱਕ ਆਰਗੂਮੈਂਟ ਦੇ ਕੇ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ:

ਇਹ ਦੋਵੇਂ ਕਦਮ **TextVectorization** ਲੇਅਰ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਸੰਭਾਲੇ ਜਾ ਸਕਦੇ ਹਨ। ਆਓ ਵੇਕਟਰਾਈਜ਼ਰ ਆਬਜੈਕਟ ਬਣਾਈਏ ਅਤੇ ਫਿਰ `adapt` ਮੈਥਡ ਨੂੰ ਕਾਲ ਕਰਕੇ ਸਾਰੇ ਟੈਕਸਟ ਵਿੱਚੋਂ ਗੁਜ਼ਰ ਕੇ ਇੱਕ ਸ਼ਬਦਾਵਲੀ ਤਿਆਰ ਕਰੀਏ:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **ਨੋਟ** ਕਿ ਅਸੀਂ ਸਿਰਫ਼ ਪੂਰੇ ਡੇਟਾਸੈੱਟ ਦੇ ਇੱਕ ਹਿੱਸੇ ਨੂੰ ਵਰਤ ਰਹੇ ਹਾਂ ਤਾਂ ਜੋ ਇੱਕ ਸ਼ਬਦਾਵਲੀ ਤਿਆਰ ਕੀਤੀ ਜਾ ਸਕੇ। ਅਸੀਂ ਇਹ ਇਸ ਲਈ ਕਰਦੇ ਹਾਂ ਤਾਂ ਕਿ ਕਾਰਜਕਾਲ ਦਾ ਸਮਾਂ ਤੇਜ਼ ਹੋਵੇ ਅਤੇ ਤੁਹਾਨੂੰ ਉਡੀਕ ਨਾ ਕਰਨੀ ਪਵੇ। ਹਾਲਾਂਕਿ, ਅਸੀਂ ਇਹ ਜੋਖਮ ਲੈ ਰਹੇ ਹਾਂ ਕਿ ਪੂਰੇ ਡੇਟਾਸੈੱਟ ਵਿੱਚੋਂ ਕੁਝ ਸ਼ਬਦ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਸ਼ਾਮਲ ਨਹੀਂ ਹੋਣਗੇ ਅਤੇ ਟ੍ਰੇਨਿੰਗ ਦੌਰਾਨ ਅਣਡਿੱਠੇ ਰਹਿ ਜਾਣਗੇ। ਇਸ ਤਰ੍ਹਾਂ, ਪੂਰੀ ਸ਼ਬਦਾਵਲੀ ਦਾ ਆਕਾਰ ਵਰਤਣਾ ਅਤੇ `adapt` ਦੌਰਾਨ ਸਾਰੇ ਡੇਟਾਸੈੱਟ ਵਿੱਚੋਂ ਗੁਜ਼ਰਨਾ ਅੰਤਿਮ ਸਹੀਤਾ ਨੂੰ ਵਧਾ ਸਕਦਾ ਹੈ, ਪਰ ਇਹ ਵਾਧਾ ਬਹੁਤ ਵੱਡਾ ਨਹੀਂ ਹੋਵੇਗਾ।

ਹੁਣ ਅਸੀਂ ਅਸਲ ਸ਼ਬਦਾਵਲੀ ਤੱਕ ਪਹੁੰਚ ਸਕਦੇ ਹਾਂ:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


ਵੇਕਟੋਰਾਈਜ਼ਰ ਦੀ ਵਰਤੋਂ ਕਰਕੇ, ਅਸੀਂ ਆਸਾਨੀ ਨਾਲ ਕਿਸੇ ਵੀ ਟੈਕਸਟ ਨੂੰ ਨੰਬਰਾਂ ਦੇ ਸੈੱਟ ਵਿੱਚ ਕੋਡ ਕਰ ਸਕਦੇ ਹਾਂ:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## ਬੈਗ-ਆਫ-ਵਰਡਸ ਟੈਕਸਟ ਪ੍ਰਤੀਨਿਧਤਾ

ਕਿਉਂਕਿ ਸ਼ਬਦ ਅਰਥ ਨੂੰ ਦਰਸਾਉਂਦੇ ਹਨ, ਕਈ ਵਾਰ ਅਸੀਂ ਸਿਰਫ਼ ਵੱਖ-ਵੱਖ ਸ਼ਬਦਾਂ ਨੂੰ ਦੇਖ ਕੇ ਟੈਕਸਟ ਦੇ ਅਰਥ ਨੂੰ ਸਮਝ ਸਕਦੇ ਹਾਂ, ਭਾਵੇਂ ਉਹ ਵਾਕ ਵਿੱਚ ਕਿਸੇ ਵੀ ਕ੍ਰਮ ਵਿੱਚ ਹੋਣ। ਉਦਾਹਰਣ ਵਜੋਂ, ਜਦੋਂ ਅਸੀਂ ਖ਼ਬਰਾਂ ਦੀ ਵਰਗੀਕਰਨ ਕਰਦੇ ਹਾਂ, ਤਾਂ *weather* ਅਤੇ *snow* ਵਰਗੇ ਸ਼ਬਦ *weather forecast* ਵੱਲ ਇਸ਼ਾਰਾ ਕਰਦੇ ਹਨ, ਜਦਕਿ *stocks* ਅਤੇ *dollar* ਵਰਗੇ ਸ਼ਬਦ *financial news* ਵੱਲ ਗਿਣੇ ਜਾਂਦੇ ਹਨ।

**ਬੈਗ-ਆਫ-ਵਰਡਸ** (BoW) ਵੇਕਟਰ ਪ੍ਰਤੀਨਿਧਤਾ ਸਭ ਤੋਂ ਸਧਾਰਨ ਅਤੇ ਸਮਝਣ ਵਿੱਚ ਆਸਾਨ ਰਵਾਇਤੀ ਵੇਕਟਰ ਪ੍ਰਤੀਨਿਧਤਾ ਹੈ। ਹਰ ਸ਼ਬਦ ਨੂੰ ਇੱਕ ਵੇਕਟਰ ਇੰਡੈਕਸ ਨਾਲ ਜੋੜਿਆ ਜਾਂਦਾ ਹੈ, ਅਤੇ ਇੱਕ ਵੇਕਟਰ ਤੱਤ ਵਿੱਚ ਦਿੱਤੇ ਗਏ ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਹਰ ਸ਼ਬਦ ਦੇ ਆਵਿਰਤੀ ਦੀ ਗਿਣਤੀ ਹੁੰਦੀ ਹੈ।

![ਇਮੇਜ ਦਿਖਾ ਰਿਹਾ ਹੈ ਕਿ ਬੈਗ-ਆਫ-ਵਰਡਸ ਵੇਕਟਰ ਪ੍ਰਤੀਨਿਧਤਾ ਮੈਮੋਰੀ ਵਿੱਚ ਕਿਵੇਂ ਦਰਸਾਈ ਜਾਂਦੀ ਹੈ।](../../../../../translated_images/pa/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: ਤੁਸੀਂ BoW ਨੂੰ ਟੈਕਸਟ ਵਿੱਚ ਵੱਖ-ਵੱਖ ਸ਼ਬਦਾਂ ਲਈ ਇੱਕ-ਹਾਟ-ਐਨਕੋਡਡ ਵੇਕਟਰਾਂ ਦੇ ਜੋੜ ਵਜੋਂ ਵੀ ਸੋਚ ਸਕਦੇ ਹੋ।

ਹੇਠਾਂ Scikit Learn ਪਾਇਥਨ ਲਾਇਬ੍ਰੇਰੀ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਬੈਗ-ਆਫ-ਵਰਡਸ ਪ੍ਰਤੀਨਿਧਤਾ ਬਣਾਉਣ ਦਾ ਇੱਕ ਉਦਾਹਰਣ ਦਿੱਤਾ ਗਿਆ ਹੈ:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

ਅਸੀਂ ਉਪਰ ਪਰिभਾਸ਼ਿਤ ਕੀਤਾ ਕੈਰਾਸ ਵੈਕਟਰਾਈਜ਼ਰ ਵੀ ਵਰਤ ਸਕਦੇ ਹਾਂ, ਹਰ ਸ਼ਬਦ ਦੀ ਗਿਣਤੀ ਨੂੰ ਇੱਕ-ਹਾਟ ਕੋਡਿੰਗ ਵਿੱਚ ਰੂਪਾਂਤਰਿਤ ਕਰਦੇ ਹੋਏ ਅਤੇ ਉਹਨਾਂ ਸਾਰੇ ਵੈਕਟਰਾਂ ਨੂੰ ਜੋੜਦੇ ਹੋਏ:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **ਨੋਟ**: ਤੁਸੀਂ ਹੈਰਾਨ ਹੋ ਸਕਦੇ ਹੋ ਕਿ ਨਤੀਜਾ ਪਿਛਲੇ ਉਦਾਹਰਨ ਤੋਂ ਵੱਖਰਾ ਹੈ। ਇਸ ਦਾ ਕਾਰਨ ਇਹ ਹੈ ਕਿ Keras ਦੇ ਉਦਾਹਰਨ ਵਿੱਚ ਵੇਕਟਰ ਦੀ ਲੰਬਾਈ ਸ਼ਬਦਾਵਲੀ ਦੇ ਆਕਾਰ ਦੇ ਬਰਾਬਰ ਹੈ, ਜੋ ਪੂਰੇ AG News ਡੇਟਾਸੈੱਟ ਤੋਂ ਬਣਾਈ ਗਈ ਸੀ, ਜਦਕਿ Scikit Learn ਦੇ ਉਦਾਹਰਨ ਵਿੱਚ ਅਸੀਂ ਸ਼ਬਦਾਵਲੀ ਨੂੰ ਸੈਂਪਲ ਟੈਕਸਟ ਤੋਂ ਤੁਰੰਤ ਬਣਾਇਆ।


## BoW ਕਲਾਸੀਫਾਇਰ ਦੀ ਟ੍ਰੇਨਿੰਗ

ਹੁਣ ਜਦੋਂ ਅਸੀਂ ਸਿੱਖ ਲਿਆ ਹੈ ਕਿ ਆਪਣੇ ਟੈਕਸਟ ਦਾ ਬੈਗ-ਆਫ-ਵਰਡਸ ਪ੍ਰਤੀਨਿਧੀ ਕਿਵੇਂ ਬਣਾਉਣਾ ਹੈ, ਆਓ ਇੱਕ ਕਲਾਸੀਫਾਇਰ ਟ੍ਰੇਨ ਕਰੀਏ ਜੋ ਇਸਨੂੰ ਵਰਤਦਾ ਹੈ। ਸਭ ਤੋਂ ਪਹਿਲਾਂ, ਸਾਨੂੰ ਆਪਣਾ ਡੇਟਾਸੈਟ ਬੈਗ-ਆਫ-ਵਰਡਸ ਪ੍ਰਤੀਨਿਧੀ ਵਿੱਚ ਤਬਦੀਲ ਕਰਨਾ ਪਵੇਗਾ। ਇਹ `map` ਫੰਕਸ਼ਨ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਹਾਸਲ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

ਹੁਣ ਆਓ ਇੱਕ ਸਧਾਰਨ ਕਲਾਸੀਫਾਇਰ ਨਿਊਰਲ ਨੈਟਵਰਕ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰੀਏ ਜੋ ਇੱਕ ਲਿਨੀਅਰ ਲੇਅਰ ਸ਼ਾਮਲ ਕਰਦਾ ਹੈ। ਇਨਪੁਟ ਸਾਈਜ਼ `vocab_size` ਹੈ, ਅਤੇ ਆਉਟਪੁਟ ਸਾਈਜ਼ ਕਲਾਸਾਂ ਦੀ ਗਿਣਤੀ (4) ਦੇ ਅਨੁਸਾਰ ਹੈ। ਕਿਉਂਕਿ ਅਸੀਂ ਇੱਕ ਕਲਾਸੀਫਿਕੇਸ਼ਨ ਟਾਸਕ ਹੱਲ ਕਰ ਰਹੇ ਹਾਂ, ਅੰਤਮ ਐਕਟੀਵੇਸ਼ਨ ਫੰਕਸ਼ਨ **softmax** ਹੈ:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


ਚਾਰ ਕਲਾਸਾਂ ਹੋਣ ਕਰਕੇ, 80% ਤੋਂ ਵੱਧ ਦੀ ਸਹੀਤਾ ਇੱਕ ਵਧੀਆ ਨਤੀਜਾ ਹੈ।

## ਇੱਕ ਨੈੱਟਵਰਕ ਵਜੋਂ ਕਲਾਸੀਫਾਇਰ ਨੂੰ ਟ੍ਰੇਨ ਕਰਨਾ

ਕਿਉਂਕਿ ਵੈਕਟੋਰਾਈਜ਼ਰ ਵੀ ਇੱਕ Keras ਲੇਅਰ ਹੈ, ਅਸੀਂ ਇੱਕ ਨੈੱਟਵਰਕ ਪਰਿਭਾਸ਼ਿਤ ਕਰ ਸਕਦੇ ਹਾਂ ਜੋ ਇਸਨੂੰ ਸ਼ਾਮਲ ਕਰਦਾ ਹੈ ਅਤੇ ਇਸਨੂੰ ਐਂਡ-ਟੂ-ਐਂਡ ਟ੍ਰੇਨ ਕਰ ਸਕਦੇ ਹਾਂ। ਇਸ ਤਰੀਕੇ ਨਾਲ ਸਾਨੂੰ ਡਾਟਾਸੈਟ ਨੂੰ `map` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਵੈਕਟੋਰਾਈਜ਼ ਕਰਨ ਦੀ ਲੋੜ ਨਹੀਂ ਹੈ, ਅਸੀਂ ਸਿਰਫ਼ ਅਸਲ ਡਾਟਾਸੈਟ ਨੂੰ ਨੈੱਟਵਰਕ ਦੇ ਇਨਪੁਟ 'ਤੇ ਪਾਸ ਕਰ ਸਕਦੇ ਹਾਂ।

> **Note**: ਸਾਨੂੰ ਫਿਰ ਵੀ ਆਪਣੇ ਡਾਟਾਸੈਟ 'ਤੇ ਮੈਪ ਲਾਗੂ ਕਰਨ ਦੀ ਲੋੜ ਹੋਵੇਗੀ ਤਾਂ ਜੋ ਡਿਕਸ਼ਨਰੀਜ਼ (ਜਿਵੇਂ ਕਿ `title`, `description` ਅਤੇ `label`) ਤੋਂ ਫੀਲਡ ਨੂੰ ਟਿਊਪਲ ਵਿੱਚ ਬਦਲਿਆ ਜਾ ਸਕੇ। ਹਾਲਾਂਕਿ, ਜਦੋਂ ਅਸੀਂ ਡਿਸਕ ਤੋਂ ਡਾਟਾ ਲੋਡ ਕਰਦੇ ਹਾਂ, ਅਸੀਂ ਪਹਿਲੇ ਹੀ ਸਹੀ ਸਟ੍ਰਕਚਰ ਵਾਲਾ ਡਾਟਾਸੈਟ ਤਿਆਰ ਕਰ ਸਕਦੇ ਹਾਂ।


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## ਬਾਈਗ੍ਰਾਮ, ਟ੍ਰਾਈਗ੍ਰਾਮ ਅਤੇ ਐਨ-ਗ੍ਰਾਮ

ਬੈਗ-ਆਫ-ਵਰਡਸ ਪਹੁੰਚ ਦੀ ਇੱਕ ਸੀਮਿਤਤਾ ਇਹ ਹੈ ਕਿ ਕੁਝ ਸ਼ਬਦ ਬਹੁ-ਸ਼ਬਦੀ ਅਭਿਵਿਅੰਜਨਾਂ ਦਾ ਹਿੱਸਾ ਹੁੰਦੇ ਹਨ। ਉਦਾਹਰਨ ਵਜੋਂ, 'ਹਾਟ ਡੌਗ' ਸ਼ਬਦ ਦਾ ਅਰਥ 'ਹਾਟ' ਅਤੇ 'ਡੌਗ' ਸ਼ਬਦਾਂ ਦੇ ਹੋਰ ਸੰਦਰਭਾਂ ਵਿੱਚੋਂ ਬਿਲਕੁਲ ਵੱਖਰਾ ਹੈ। ਜੇਕਰ ਅਸੀਂ 'ਹਾਟ' ਅਤੇ 'ਡੌਗ' ਸ਼ਬਦਾਂ ਨੂੰ ਹਮੇਸ਼ਾ ਇੱਕੋ ਜਿਹੇ ਵੇਕਟਰਾਂ ਨਾਲ ਦਰਸਾਉਂਦੇ ਹਾਂ, ਤਾਂ ਇਹ ਸਾਡੇ ਮਾਡਲ ਨੂੰ ਗੁੰਝਲ ਵਿੱਚ ਪਾ ਸਕਦਾ ਹੈ।

ਇਸ ਸਮੱਸਿਆ ਨੂੰ ਹੱਲ ਕਰਨ ਲਈ, **ਐਨ-ਗ੍ਰਾਮ ਪ੍ਰਤੀਨਿਧੀਆਂ** ਨੂੰ ਦਸਤਾਵੇਜ਼ ਵਰਗੀਕਰਨ ਦੇ ਤਰੀਕਿਆਂ ਵਿੱਚ ਅਕਸਰ ਵਰਤਿਆ ਜਾਂਦਾ ਹੈ, ਜਿੱਥੇ ਹਰ ਸ਼ਬਦ, ਦੋ-ਸ਼ਬਦ ਜਾਂ ਤਿੰਨ-ਸ਼ਬਦ ਦੀ ਆਵ੍ਰਿਤੀ ਵਰਗੀਕਰਨ ਮਾਡਲਾਂ ਨੂੰ ਸਿਖਾਉਣ ਲਈ ਇੱਕ ਲਾਭਦਾਇਕ ਵਿਸ਼ੇਸ਼ਤਾ ਹੁੰਦੀ ਹੈ। ਬਾਈਗ੍ਰਾਮ ਪ੍ਰਤੀਨਿਧੀਆਂ ਵਿੱਚ, ਉਦਾਹਰਨ ਵਜੋਂ, ਅਸੀਂ ਮੂਲ ਸ਼ਬਦਾਂ ਦੇ ਨਾਲ ਨਾਲ ਸਾਰੇ ਸ਼ਬਦ ਜੋੜਾਂ ਨੂੰ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਸ਼ਾਮਲ ਕਰਾਂਗੇ।

ਹੇਠਾਂ ਦਿੱਤਾ ਇੱਕ ਉਦਾਹਰਨ ਹੈ ਕਿ ਕਿਵੇਂ Scikit Learn ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਬਾਈਗ੍ਰਾਮ ਬੈਗ-ਆਫ-ਵਰਡ ਪ੍ਰਤੀਨਿਧੀ ਬਣਾਈ ਜਾ ਸਕਦੀ ਹੈ:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-gram ਪਹੁੰਚ ਦਾ ਮੁੱਖ ਨੁਕਸਾਨ ਇਹ ਹੈ ਕਿ ਸ਼ਬਦਾਵਲੀ ਦਾ ਆਕਾਰ ਬਹੁਤ ਤੇਜ਼ੀ ਨਾਲ ਵਧਣਾ ਸ਼ੁਰੂ ਹੋ ਜਾਂਦਾ ਹੈ। ਅਮਲ ਵਿੱਚ, ਸਾਨੂੰ n-gram ਪ੍ਰਸਤੁਤੀ ਨੂੰ ਇੱਕ ਮਾਪ ਘਟਾਉਣ ਦੀ ਤਕਨੀਕ ਨਾਲ ਜੋੜਨਾ ਪੈਂਦਾ ਹੈ, ਜਿਵੇਂ ਕਿ *embeddings*, ਜਿਸ ਬਾਰੇ ਅਸੀਂ ਅਗਲੇ ਯੂਨਿਟ ਵਿੱਚ ਚਰਚਾ ਕਰਾਂਗੇ।

**AG News** ਡੇਟਾਸੈੱਟ ਵਿੱਚ n-gram ਪ੍ਰਸਤੁਤੀ ਨੂੰ ਵਰਤਣ ਲਈ, ਸਾਨੂੰ `TextVectorization` ਕੰਸਟਰਕਟਰ ਨੂੰ `ngrams` ਪੈਰਾਮੀਟਰ ਪਾਸ ਕਰਨਾ ਪਵੇਗਾ। ਇੱਕ bigram ਸ਼ਬਦਾਵਲੀ ਦੀ ਲੰਬਾਈ **ਕਾਫ਼ੀ ਵੱਡੀ** ਹੁੰਦੀ ਹੈ, ਸਾਡੇ ਕੇਸ ਵਿੱਚ ਇਹ 1.3 ਮਿਲੀਅਨ ਤੋਂ ਵੱਧ ਟੋਕਨ ਹਨ! ਇਸ ਲਈ, ਇਹ ਸਮਝਦਾਰੀ ਵਾਲੀ ਗੱਲ ਹੈ ਕਿ bigram ਟੋਕਨ ਨੂੰ ਵੀ ਕਿਸੇ ਵਾਜਬ ਗਿਣਤੀ ਤੱਕ ਸੀਮਿਤ ਕੀਤਾ ਜਾਵੇ।

ਅਸੀਂ ਉਪਰ ਦਿੱਤੇ ਗਏ ਕੋਡ ਨੂੰ ਵਰਤ ਕੇ ਕਲਾਸੀਫਾਇਰ ਨੂੰ ਟ੍ਰੇਨ ਕਰ ਸਕਦੇ ਹਾਂ, ਪਰ ਇਹ ਯਾਦਦਾਸ਼ਤ ਦੀ ਦ੍ਰਿਸ਼ਟੀ ਤੋਂ ਬਹੁਤ ਅਪਰਭਾਵਸ਼ਾਲੀ ਹੋਵੇਗਾ। ਅਗਲੇ ਯੂਨਿਟ ਵਿੱਚ, ਅਸੀਂ embeddings ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹੋਏ bigram ਕਲਾਸੀਫਾਇਰ ਨੂੰ ਟ੍ਰੇਨ ਕਰਾਂਗੇ। ਇਸ ਦੌਰਾਨ, ਤੁਸੀਂ ਇਸ ਨੋਟਬੁੱਕ ਵਿੱਚ bigram ਕਲਾਸੀਫਾਇਰ ਟ੍ਰੇਨਿੰਗ ਨਾਲ ਪ੍ਰਯੋਗ ਕਰ ਸਕਦੇ ਹੋ ਅਤੇ ਵੇਖ ਸਕਦੇ ਹੋ ਕਿ ਕੀ ਤੁਸੀਂ ਵਧੇਰੇ ਸਹੀ ਨਤੀਜੇ ਪ੍ਰਾਪਤ ਕਰ ਸਕਦੇ ਹੋ।


## ਬੋਵ ਵੈਕਟਰਾਂ ਦੀ ਆਟੋਮੈਟਿਕ ਗਿਣਤੀ

ਉਪਰੋਕਤ ਉਦਾਹਰਨ ਵਿੱਚ ਅਸੀਂ ਬੋਵ ਵੈਕਟਰਾਂ ਨੂੰ ਹੱਥੋਂ ਗਿਣਿਆ ਸੀ, ਜਿੱਥੇ ਅਸੀਂ ਵਿਅਕਤੀਗਤ ਸ਼ਬਦਾਂ ਦੇ ਇੱਕ-ਹਾਟ ਐਨਕੋਡਿੰਗ ਨੂੰ ਜੋੜਿਆ ਸੀ। ਹਾਲਾਂਕਿ, TensorFlow ਦੇ ਨਵੇਂ ਵਰਜਨ ਵਿੱਚ ਅਸੀਂ ਬੋਵ ਵੈਕਟਰਾਂ ਨੂੰ ਆਟੋਮੈਟਿਕ ਤੌਰ 'ਤੇ ਗਿਣ ਸਕਦੇ ਹਾਂ, ਜਦੋਂ ਅਸੀਂ `output_mode='count` ਪੈਰਾਮੀਟਰ ਨੂੰ ਵੈਕਟੋਰਾਈਜ਼ਰ ਕੰਸਟਰੱਕਟਰ ਵਿੱਚ ਪਾਸ ਕਰਦੇ ਹਾਂ। ਇਸ ਨਾਲ ਸਾਡਾ ਮਾਡਲ ਡਿਫਾਈਨ ਕਰਨਾ ਅਤੇ ਟ੍ਰੇਨ ਕਰਨਾ ਕਾਫ਼ੀ ਆਸਾਨ ਹੋ ਜਾਂਦਾ ਹੈ:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## ਟਰਮ ਫ੍ਰੀਕਵੈਂਸੀ - ਇਨਵਰਸ ਡੌਕਯੂਮੈਂਟ ਫ੍ਰੀਕਵੈਂਸੀ (TF-IDF)

BoW ਪ੍ਰਸਤੁਤੀ ਵਿੱਚ, ਸ਼ਬਦਾਂ ਦੀ ਗਿਣਤੀ ਨੂੰ ਇੱਕੋ ਤਰੀਕੇ ਨਾਲ ਵਜ਼ਨ ਦਿੱਤਾ ਜਾਂਦਾ ਹੈ, ਭਾਵੇਂ ਸ਼ਬਦ ਦੀ ਕਿਸਮ ਜੋ ਮਰਜ਼ੀ ਹੋਵੇ। ਹਾਲਾਂਕਿ, ਇਹ ਸਪਸ਼ਟ ਹੈ ਕਿ ਆਮ ਸ਼ਬਦ ਜਿਵੇਂ ਕਿ *a* ਅਤੇ *in* ਵਰਗੇ ਸ਼ਬਦ ਵਰਤੋਂ ਲਈ ਵਿਸ਼ੇਸ਼ ਸ਼ਬਦਾਂ ਦੇ ਮੁਕਾਬਲੇ ਘੱਟ ਮਹੱਤਵਪੂਰਨ ਹੁੰਦੇ ਹਨ। ਜ਼ਿਆਦਾਤਰ NLP ਕੰਮਾਂ ਵਿੱਚ ਕੁਝ ਸ਼ਬਦ ਹੋਰਾਂ ਨਾਲੋਂ ਜ਼ਿਆਦਾ ਮਹੱਤਵਪੂਰਨ ਹੁੰਦੇ ਹਨ।

**TF-IDF** ਦਾ ਮਤਲਬ ਹੈ **ਟਰਮ ਫ੍ਰੀਕਵੈਂਸੀ - ਇਨਵਰਸ ਡੌਕਯੂਮੈਂਟ ਫ੍ਰੀਕਵੈਂਸੀ**। ਇਹ ਬੈਗ-ਆਫ-ਵਰਡਜ਼ ਦੀ ਇੱਕ ਵੈਰੀਏਸ਼ਨ ਹੈ, ਜਿਸ ਵਿੱਚ ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਸ਼ਬਦ ਦੀ ਮੌਜੂਦਗੀ ਨੂੰ ਦਰਸਾਉਣ ਵਾਲੇ ਬਾਈਨਰੀ 0/1 ਮੁੱਲ ਦੀ ਥਾਂ ਇੱਕ ਫਲੋਟਿੰਗ-ਪੌਇੰਟ ਮੁੱਲ ਵਰਤਿਆ ਜਾਂਦਾ ਹੈ, ਜੋ ਸ਼ਬਦ ਦੀ ਕੌਰਪਸ ਵਿੱਚ ਆਵਿਰਤੀ ਨਾਲ ਸੰਬੰਧਿਤ ਹੁੰਦਾ ਹੈ।

ਵਧੇਰੇ ਰਸਮੀ ਤੌਰ 'ਤੇ, ਦਸਤਾਵੇਜ਼ $j$ ਵਿੱਚ ਸ਼ਬਦ $i$ ਦਾ ਵਜ਼ਨ $w_{ij}$ ਇਸ ਤਰ੍ਹਾਂ ਪਰਿਭਾਸ਼ਿਤ ਕੀਤਾ ਜਾਂਦਾ ਹੈ:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ਜਿੱਥੇ
* $tf_{ij}$ $j$ ਵਿੱਚ $i$ ਦੀ ਗਿਣਤੀ ਹੈ, ਅਰਥਾਤ ਉਹ BoW ਮੁੱਲ ਜੋ ਅਸੀਂ ਪਹਿਲਾਂ ਵੇਖਿਆ ਸੀ
* $N$ ਕਲੈਕਸ਼ਨ ਵਿੱਚ ਦਸਤਾਵੇਜ਼ਾਂ ਦੀ ਗਿਣਤੀ ਹੈ
* $df_i$ ਪੂਰੀ ਕਲੈਕਸ਼ਨ ਵਿੱਚ ਉਹ ਦਸਤਾਵੇਜ਼ਾਂ ਦੀ ਗਿਣਤੀ ਹੈ ਜਿਨ੍ਹਾਂ ਵਿੱਚ ਸ਼ਬਦ $i$ ਸ਼ਾਮਲ ਹੈ

TF-IDF ਮੁੱਲ $w_{ij}$ ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਸ਼ਬਦ ਦੀ ਗਿਣਤੀ ਦੇ ਅਨੁਪਾਤ ਵਿੱਚ ਵਧਦਾ ਹੈ ਅਤੇ ਕੌਰਪਸ ਵਿੱਚ ਉਹ ਦਸਤਾਵੇਜ਼ਾਂ ਦੀ ਗਿਣਤੀ ਨਾਲ ਸੰਬੰਧਿਤ ਹੁੰਦਾ ਹੈ ਜਿਨ੍ਹਾਂ ਵਿੱਚ ਉਹ ਸ਼ਬਦ ਸ਼ਾਮਲ ਹੈ। ਇਹ ਇਸ ਗੱਲ ਨੂੰ ਠੀਕ ਕਰਨ ਵਿੱਚ ਮਦਦ ਕਰਦਾ ਹੈ ਕਿ ਕੁਝ ਸ਼ਬਦ ਹੋਰਾਂ ਨਾਲੋਂ ਵੱਧ ਵਾਰ ਆਉਂਦੇ ਹਨ। ਉਦਾਹਰਣ ਲਈ, ਜੇਕਰ ਸ਼ਬਦ ਕਲੈਕਸ਼ਨ ਵਿੱਚ *ਹਰ* ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਆਉਂਦਾ ਹੈ, ਤਾਂ $df_i=N$, ਅਤੇ $w_{ij}=0$, ਅਤੇ ਉਹ ਸ਼ਬਦ ਪੂਰੀ ਤਰ੍ਹਾਂ ਨਜ਼ਰਅੰਦਾਜ਼ ਕਰ ਦਿੱਤੇ ਜਾਣਗੇ।

ਤੁਸੀਂ ਆਸਾਨੀ ਨਾਲ Scikit Learn ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਟੈਕਸਟ ਦੀ TF-IDF ਵੈਕਟਰਾਈਜ਼ੇਸ਼ਨ ਬਣਾ ਸਕਦੇ ਹੋ:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

ਕੇਰਾਸ ਵਿੱਚ, `TextVectorization` ਲੇਅਰ `output_mode='tf-idf'` ਪੈਰਾਮੀਟਰ ਪਾਸ ਕਰਕੇ ਸਵੈਚਾਲਿਤ ਤੌਰ 'ਤੇ TF-IDF ਫ੍ਰੀਕਵੈਂਸੀ ਦੀ ਗਣਨਾ ਕਰ ਸਕਦੀ ਹੈ। ਆਓ, ਉਪਰੋਕਤ ਵਰਤੇ ਗਏ ਕੋਡ ਨੂੰ ਦੁਹਰਾਈਏ ਤਾਂ ਜੋ ਦੇਖ ਸਕੀਏ ਕਿ TF-IDF ਵਰਤਣ ਨਾਲ ਸ਼ੁੱਧਤਾ ਵਿੱਚ ਵਾਧਾ ਹੁੰਦਾ ਹੈ ਜਾਂ ਨਹੀਂ:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## ਨਤੀਜਾ

ਹਾਲਾਂਕਿ TF-IDF ਪ੍ਰਸਤੁਤੀਆਂ ਵੱਖ-ਵੱਖ ਸ਼ਬਦਾਂ ਨੂੰ ਆਵ੍ਰਿਤੀ ਵਜ਼ਨ ਪ੍ਰਦਾਨ ਕਰਦੀਆਂ ਹਨ, ਇਹ ਅਰਥ ਜਾਂ ਕ੍ਰਮ ਨੂੰ ਦਰਸਾਉਣ ਵਿੱਚ ਅਸਮਰਥ ਹਨ। ਪ੍ਰਸਿੱਧ ਭਾਸ਼ਾ ਵਿਗਿਆਨੀ J. R. Firth ਨੇ 1935 ਵਿੱਚ ਕਿਹਾ ਸੀ, "ਸ਼ਬਦ ਦਾ ਪੂਰਾ ਅਰਥ ਹਮੇਸ਼ਾ ਸੰਦਰਭਕ ਹੁੰਦਾ ਹੈ, ਅਤੇ ਸੰਦਰਭ ਤੋਂ ਬਿਨਾਂ ਅਰਥ ਦਾ ਕੋਈ ਅਧਿਐਨ ਗੰਭੀਰਤਾ ਨਾਲ ਨਹੀਂ ਲਿਆ ਜਾ ਸਕਦਾ।" ਅਸੀਂ ਇਸ ਕੋਰਸ ਵਿੱਚ ਅਗਲੇ ਹਿੱਸੇ ਵਿੱਚ ਭਾਸ਼ਾ ਮਾਡਲਿੰਗ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਟੈਕਸਟ ਤੋਂ ਸੰਦਰਭਕ ਜਾਣਕਾਰੀ ਕਿਵੇਂ ਕੈਪਚਰ ਕਰਨੀ ਹੈ, ਇਹ ਸਿੱਖਾਂਗੇ।



---

**ਅਸਵੀਕਾਰਨਾ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੱਜੇਪਣ ਹੋ ਸਕਦੇ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼, ਜੋ ਇਸਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਹੈ, ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
